In [1]:
import os
import numpy as np
import cv2 as cv
import h5py
import matplotlib.pyplot as plt
import keras
from keras.layers import Conv2D, MaxPool2D, CuDNNGRU, GlobalMaxPool2D, Reshape, \
concatenate, Input, TimeDistributed, Dense, BatchNormalization, SpatialDropout2D, SpatialDropout1D, Dropout, GlobalAvgPool2D, Flatten
from keras import Model
from keras.applications import Xception
import keras.backend as k
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


## First Cache all the images into your Disk

In [3]:
vidPath=r".\data\train\train.mp4"
imgStoragePath=r".\data\train\IMG"

IMGList=[]
def write_frames():
    op_flows=[]
    frames=[]
    count=0
    vidCap=cv.VideoCapture(vidPath)
    success=True
    while success:
        if(count%1000==0) and count >0:
            print(count)
        success,frame1=vidCap.read()
        if (success == True):
            frame1=cv.cvtColor(frame1,cv.COLOR_BGR2RGB)
            frame1=cv.resize(frame1, (0,0), fx=0.5, fy=0.5) 
            name=imgStoragePath
            filename=str(count)+r".jpg"
            IMGList.append(os.path.join(name, filename))
            cv.imwrite(os.path.join(name, filename),frame1)  
            count+=1
        else:
            print("video reading completed")
            continue
    return count

Total_frames=write_frames()

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
video reading completed


In [4]:
print(Total_frames,' :Total Frames')

print('Size of FileList: ',len(IMGList))


20400  :Total Frames
Size of FileList:  20400


In [47]:
hdf5_path1 = r".\data\train\train1.hdf5"
hdf5_path2 = r".\data\train\train2.hdf5"
hdf5_path3 = r".\data\train\train3.hdf5"
hdf5_path4 = r".\data\train\train4.hdf5"

resize_size=(240, 320, 2)
resize_frame_size=(240, 320, 3)

'''Only use to reset dataset'''
with h5py.File(hdf5_path1, "w") as f:
    f.create_dataset("frames", shape = (1, resize_frame_size[0], resize_frame_size[1], resize_frame_size[2]),
                    maxshape = (None, resize_frame_size[0], resize_frame_size[1], resize_frame_size[2]), 
                    chunks = (1, resize_frame_size[0], resize_frame_size[1], resize_frame_size[2]))
    f.create_dataset("speeds", shape = (1,1), maxshape = (None,1))
f.close()



'''Only use to reset dataset'''
with h5py.File(hdf5_path2, "w") as f:
    f.create_dataset("frames", shape = (1, resize_frame_size[0], resize_frame_size[1], resize_frame_size[2]),
                    maxshape = (None, resize_frame_size[0], resize_frame_size[1], resize_frame_size[2]), 
                    chunks = (1, resize_frame_size[0], resize_frame_size[1], resize_frame_size[2]))
    f.create_dataset("speeds", shape = (1,1), maxshape = (None,1))
f.close()


'''Only use to reset dataset'''
with h5py.File(hdf5_path3, "w") as f:
    f.create_dataset("frames", shape = (1, resize_frame_size[0], resize_frame_size[1], resize_frame_size[2]),
                    maxshape = (None, resize_frame_size[0], resize_frame_size[1], resize_frame_size[2]), 
                    chunks = (1, resize_frame_size[0], resize_frame_size[1], resize_frame_size[2]))
    f.create_dataset("speeds", shape = (1,1), maxshape = (None,1))
f.close()


'''Only use to reset dataset'''
with h5py.File(hdf5_path4, "w") as f:
    f.create_dataset("frames", shape = (1, resize_frame_size[0], resize_frame_size[1], resize_frame_size[2]),
                    maxshape = (None, resize_frame_size[0], resize_frame_size[1], resize_frame_size[2]), 
                    chunks = (1, resize_frame_size[0], resize_frame_size[1], resize_frame_size[2]))
    f.create_dataset("speeds", shape = (1,1), maxshape = (None,1))
f.close()

In [48]:
def write_hdf5(hdf5_path, frames, speeds):
    with h5py.File(hdf5_path) as f:
        print(len(frames), len(speeds))
        print(f["frames"], f["speeds"])
        f["frames"].resize(f["frames"].len() + len(frames), axis = 0)
        f["speeds"].resize(f["speeds"].len() + len(speeds), axis = 0)
        f["frames"][-len(frames):] = frames
        f["speeds"][-len(speeds):] = speeds


### We are preproceesing the video to combine two frames into one

In [49]:
def opticalFlowDense(image_current,image_next):
    image_current=np.array(image_current)
    image_next=np.array(image_current)
    
    #convert to grayscale
    gray_current=cv.cvtColor(image_current,cv.COLOR_RGB2GRAY)
    gray_next=cv.cvtColor(image_next,cv.COLOR_RGB2GRAY)
    flow=cv.calcOpticalFlowFarneback(gray_current,gray_next,None,0.5,1,15,2,5,1.3,0)
    return flow


def augment(image_current, image_next):
    brightness=np.random.uniform(0.5,1.5)
    image_current_1 = cv.cvtColor(image_current,cv.COLOR_RGB2HSV)
    image_current_1[:,:,2] = image_current_1[:,:,2]*brightness
    
    image_next_1 = cv.cvtColor(image_next,cv.COLOR_RGB2HSV)
    image_next_1[:,:,2] = image_next_1[:,:,2]*brightness
    
    image_current_1 = cv.cvtColor(image_current_1,cv.COLOR_HSV2RGB)
    image_next_1 = cv.cvtColor(image_next_1,cv.COLOR_HSV2RGB)
    #image_current_1 = cv.normalize(image_current_1, None, alpha=0, beta=1, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)
    #image_next_1 = cv.normalize(image_next_1, None, alpha=0, beta=1, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)
    
    return image_current_1,image_next_1

def flipimage(image_current, image_next):
    image_current=cv.flip(image_current, 1)
    image_next=cv.flip(image_next,1)
    return image_current,image_next

def augment(image_current):
    brightness=np.random.uniform(0.5,1.5)
    image_current_1 = cv.cvtColor(image_current,cv.COLOR_RGB2HSV)
    image_current_1[:,:,2] = image_current_1[:,:,2]*brightness
    
    image_current_1 = cv.cvtColor(image_current_1,cv.COLOR_HSV2RGB)
    
    return image_current_1

def flipimage(image_current):
    image_current=cv.flip(image_current, 1)
    return image_current


In [50]:
import csv

speeds=[]
with open(r".\data\train\train.txt") as csvfile:
    reader=csv.reader(csvfile)
    for speed in reader:
        speeds.append(speed)
        
print(len(speeds))

20400


In [51]:
Sequence_size=len(speeds)
print(Sequence_size, ': Sequence Size')

Batch_size=100
print(Batch_size,' : BatchSize')

Num_batches=int(np.ceil(Sequence_size/Batch_size))
print(Num_batches,' : Number of Batches')


All_indices=[item for item in range(1,Sequence_size)]
print(All_indices[-1],'Last elem')
print(All_indices[0],'First elem')


#print(All_indices,'All indices')

def generate_indices(Batch_num):
    return All_indices[Batch_num*Batch_size:(Batch_num+1)*Batch_size]

print(generate_indices(203))

print(generate_indices(0))

20400 : Sequence Size
100  : BatchSize
204  : Number of Batches
20399 Last elem
1 First elem
[20301, 20302, 20303, 20304, 20305, 20306, 20307, 20308, 20309, 20310, 20311, 20312, 20313, 20314, 20315, 20316, 20317, 20318, 20319, 20320, 20321, 20322, 20323, 20324, 20325, 20326, 20327, 20328, 20329, 20330, 20331, 20332, 20333, 20334, 20335, 20336, 20337, 20338, 20339, 20340, 20341, 20342, 20343, 20344, 20345, 20346, 20347, 20348, 20349, 20350, 20351, 20352, 20353, 20354, 20355, 20356, 20357, 20358, 20359, 20360, 20361, 20362, 20363, 20364, 20365, 20366, 20367, 20368, 20369, 20370, 20371, 20372, 20373, 20374, 20375, 20376, 20377, 20378, 20379, 20380, 20381, 20382, 20383, 20384, 20385, 20386, 20387, 20388, 20389, 20390, 20391, 20392, 20393, 20394, 20395, 20396, 20397, 20398, 20399]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 5

In [52]:
def make_op_flow_video1():
    count=0
    for batch in range(0,Num_batches):
        #print(generate_indices(batch))
        indexes=generate_indices(batch)
        frames=[]
        speed=[]
        for index in indexes:
            #print([indexes[i],indexes[i+1]])
            
            if(count%1000==0) and count >0:
                print(count)
            
            frame2=cv.imread(IMGList[index])
            frame2=cv.cvtColor(frame2,cv.COLOR_BGR2RGB)
            
            frame2=augment(frame2)
            frame2=flipimage(frame2)
            
            
            frames.append(np.array(frame2))
            speed.append(speeds[index])
            count+=1
        
        frames=np.array(frames,dtype=np.uint8)
        speed=np.array(speed)
        speed=speed.astype(float)
        write_hdf5(hdf5_path1, frames, speed)
        
       
    print("Completed Video Processing")
    return count

def make_op_flow_video2():
    count=0
    for batch in range(0,Num_batches):
        #print(generate_indices(batch))
        indexes=generate_indices(batch)
        frames=[]
        speed=[]
        for index in indexes:
            #print([indexes[i],indexes[i+1]])
            
            if(count%1000==0) and count >0:
                print(count)
            
            frame2=cv.imread(IMGList[index])
            frame2=cv.cvtColor(frame2,cv.COLOR_BGR2RGB)
            
            frame2=flipimage(frame2)
            
            
            frames.append(np.array(frame2))
            speed.append(speeds[index])
            count+=1
        
        frames=np.array(frames,dtype=np.uint8)
        speed=np.array(speed)
        speed=speed.astype(float)
        write_hdf5(hdf5_path2, frames, speed)
        
       
    print("Completed Video Processing")
    return count


def make_op_flow_video3():
    count=0
    for batch in range(0,Num_batches):
        #print(generate_indices(batch))
        indexes=generate_indices(batch)
        frames=[]
        speed=[]
        for index in indexes:
            #print([indexes[i],indexes[i+1]])
            
            if(count%1000==0) and count >0:
                print(count)
            
            frame2=cv.imread(IMGList[index])
            frame2=cv.cvtColor(frame2,cv.COLOR_BGR2RGB)
            
            frame2=augment(frame2)
            
            
            frames.append(np.array(frame2))
            speed.append(speeds[index])
            count+=1
        
        frames=np.array(frames,dtype=np.uint8)
        speed=np.array(speed)
        speed=speed.astype(float)
        write_hdf5(hdf5_path3, frames, speed)
        
       
    print("Completed Video Processing")
    return count
    
        
    
    print("Completed Video Processing")
    return count


def make_op_flow_video4():
    count=0
    for batch in range(0,Num_batches):
        #print(generate_indices(batch))
        indexes=generate_indices(batch)
        frames=[]
        speed=[]
        for index in indexes:
            #print([indexes[i],indexes[i+1]])
            
            if(count%1000==0) and count >0:
                print(count)
            
            frame2=cv.imread(IMGList[index])
            frame2=cv.cvtColor(frame2,cv.COLOR_BGR2RGB)
            
            frames.append(np.array(frame2))
            speed.append(speeds[index])
            count+=1
        
        frames=np.array(frames,dtype=np.uint8)
        speed=np.array(speed)
        speed=speed.astype(float)
        write_hdf5(hdf5_path4, frames, speed)
        
       
    print("Completed Video Processing")
    return count

### Getting all things in order

In [53]:
print("Starting Augmentation")

print("1: Both Flip and Brightness")
count=make_op_flow_video1()
print("Complete: ",count)

print('check')

Starting Augmentation
1: Both Flip and Brightness


C:\Users\Nihar\miniconda3\envs\TensorFlow-GPU\lib\site-packages\ipykernel_launcher.py:2: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  


100 100
<HDF5 dataset "frames": shape (1, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (1, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (101, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (101, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (201, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (201, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (301, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (301, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (401, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (401, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (501, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (501, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (601, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (601, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (701, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (701, 1), type "<f4">
100 100
<HDF5 dataset "frame

100 100
<HDF5 dataset "frames": shape (6701, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (6701, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (6801, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (6801, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (6901, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (6901, 1), type "<f4">
7000
100 100
<HDF5 dataset "frames": shape (7001, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (7001, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (7101, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (7101, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (7201, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (7201, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (7301, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (7301, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (7401, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (7401, 1), type "<f4">
100

100 100
<HDF5 dataset "frames": shape (13301, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (13301, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (13401, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (13401, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (13501, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (13501, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (13601, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (13601, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (13701, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (13701, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (13801, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (13801, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (13901, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (13901, 1), type "<f4">
14000
100 100
<HDF5 dataset "frames": shape (14001, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (14001, 1)

100 100
<HDF5 dataset "frames": shape (19801, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (19801, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (19901, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (19901, 1), type "<f4">
20000
100 100
<HDF5 dataset "frames": shape (20001, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (20001, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (20101, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (20101, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (20201, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (20201, 1), type "<f4">
99 99
<HDF5 dataset "frames": shape (20301, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (20301, 1), type "<f4">
Completed Video Processing
Complete:  20399
check


In [54]:
print("Starting Augmentation")

print(": Flip")
count=make_op_flow_video2()
print("Complete: ",count)


Starting Augmentation
: Flip


C:\Users\Nihar\miniconda3\envs\TensorFlow-GPU\lib\site-packages\ipykernel_launcher.py:2: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  


100 100
<HDF5 dataset "frames": shape (1, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (1, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (101, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (101, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (201, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (201, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (301, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (301, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (401, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (401, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (501, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (501, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (601, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (601, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (701, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (701, 1), type "<f4">
100 100
<HDF5 dataset "frame

100 100
<HDF5 dataset "frames": shape (6801, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (6801, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (6901, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (6901, 1), type "<f4">
7000
100 100
<HDF5 dataset "frames": shape (7001, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (7001, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (7101, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (7101, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (7201, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (7201, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (7301, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (7301, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (7401, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (7401, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (7501, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (7501, 1), type "<f4">
100

100 100
<HDF5 dataset "frames": shape (13401, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (13401, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (13501, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (13501, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (13601, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (13601, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (13701, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (13701, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (13801, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (13801, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (13901, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (13901, 1), type "<f4">
14000
100 100
<HDF5 dataset "frames": shape (14001, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (14001, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (14101, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (14101, 1)

100 100
<HDF5 dataset "frames": shape (19901, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (19901, 1), type "<f4">
20000
100 100
<HDF5 dataset "frames": shape (20001, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (20001, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (20101, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (20101, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (20201, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (20201, 1), type "<f4">
99 99
<HDF5 dataset "frames": shape (20301, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (20301, 1), type "<f4">
Completed Video Processing
Complete:  20399


In [55]:
print("Starting Augmentation")

print("3: Brightness")
count=make_op_flow_video3()
print("Complete: ",count)


Starting Augmentation
3: Brightness


C:\Users\Nihar\miniconda3\envs\TensorFlow-GPU\lib\site-packages\ipykernel_launcher.py:2: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  


100 100
<HDF5 dataset "frames": shape (1, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (1, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (101, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (101, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (201, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (201, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (301, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (301, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (401, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (401, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (501, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (501, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (601, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (601, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (701, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (701, 1), type "<f4">
100 100
<HDF5 dataset "frame

100 100
<HDF5 dataset "frames": shape (6701, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (6701, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (6801, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (6801, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (6901, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (6901, 1), type "<f4">
7000
100 100
<HDF5 dataset "frames": shape (7001, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (7001, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (7101, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (7101, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (7201, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (7201, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (7301, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (7301, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (7401, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (7401, 1), type "<f4">
100

100 100
<HDF5 dataset "frames": shape (13301, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (13301, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (13401, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (13401, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (13501, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (13501, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (13601, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (13601, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (13701, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (13701, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (13801, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (13801, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (13901, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (13901, 1), type "<f4">
14000
100 100
<HDF5 dataset "frames": shape (14001, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (14001, 1)

100 100
<HDF5 dataset "frames": shape (19801, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (19801, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (19901, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (19901, 1), type "<f4">
20000
100 100
<HDF5 dataset "frames": shape (20001, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (20001, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (20101, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (20101, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (20201, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (20201, 1), type "<f4">
99 99
<HDF5 dataset "frames": shape (20301, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (20301, 1), type "<f4">
Completed Video Processing
Complete:  20399


In [56]:
print("Starting Augmentation")

print("4: None")
count=make_op_flow_video4()
print("Complete: ",count)


Starting Augmentation
4: None


C:\Users\Nihar\miniconda3\envs\TensorFlow-GPU\lib\site-packages\ipykernel_launcher.py:2: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  


100 100
<HDF5 dataset "frames": shape (1, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (1, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (101, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (101, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (201, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (201, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (301, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (301, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (401, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (401, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (501, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (501, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (601, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (601, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (701, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (701, 1), type "<f4">
100 100
<HDF5 dataset "frame

100 100
<HDF5 dataset "frames": shape (6701, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (6701, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (6801, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (6801, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (6901, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (6901, 1), type "<f4">
7000
100 100
<HDF5 dataset "frames": shape (7001, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (7001, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (7101, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (7101, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (7201, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (7201, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (7301, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (7301, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (7401, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (7401, 1), type "<f4">
100

100 100
<HDF5 dataset "frames": shape (13301, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (13301, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (13401, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (13401, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (13501, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (13501, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (13601, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (13601, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (13701, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (13701, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (13801, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (13801, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (13901, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (13901, 1), type "<f4">
14000
100 100
<HDF5 dataset "frames": shape (14001, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (14001, 1)

100 100
<HDF5 dataset "frames": shape (19801, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (19801, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (19901, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (19901, 1), type "<f4">
20000
100 100
<HDF5 dataset "frames": shape (20001, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (20001, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (20101, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (20101, 1), type "<f4">
100 100
<HDF5 dataset "frames": shape (20201, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (20201, 1), type "<f4">
99 99
<HDF5 dataset "frames": shape (20301, 240, 320, 3), type "<f4"> <HDF5 dataset "speeds": shape (20301, 1), type "<f4">
Completed Video Processing
Complete:  20399


## Opflow Data 

In [15]:
hdf5_path1 = r".\data\train\op1.hdf5"
hdf5_path2 = r".\data\train\op2.hdf5"
hdf5_path3 = r".\data\train\op3.hdf5"
hdf5_path4 = r".\data\train\op4.hdf5"

resize_size=(240, 320, 2)
resize_frame_size=(240, 320, 3)

'''Only use to reset dataset'''
with h5py.File(hdf5_path1, "w") as f:
    f.create_dataset("op_flow", shape = (1, resize_size[0], resize_size[1], resize_size[2]),
                    maxshape = (None, resize_size[0], resize_size[1], resize_size[2]), 
                    chunks = (1, resize_size[0], resize_size[1], resize_size[2]))
f.close()

with h5py.File(hdf5_path2, "w") as f:
    f.create_dataset("op_flow", shape = (1, resize_size[0], resize_size[1], resize_size[2]),
                    maxshape = (None, resize_size[0], resize_size[1], resize_size[2]), 
                    chunks = (1, resize_size[0], resize_size[1], resize_size[2]))
f.close()

with h5py.File(hdf5_path3, "w") as f:
    f.create_dataset("op_flow", shape = (1, resize_size[0], resize_size[1], resize_size[2]),
                    maxshape = (None, resize_size[0], resize_size[1], resize_size[2]), 
                    chunks = (1, resize_size[0], resize_size[1], resize_size[2]))
f.close()

with h5py.File(hdf5_path4, "w") as f:
    f.create_dataset("op_flow", shape = (1, resize_size[0], resize_size[1], resize_size[2]),
                    maxshape = (None, resize_size[0], resize_size[1], resize_size[2]), 
                    chunks = (1, resize_size[0], resize_size[1], resize_size[2]))
f.close()





In [16]:
def write_hdf5(hdf5_path, op_flows):
    with h5py.File(hdf5_path) as f:
        print( len(op_flows))
        print( f["op_flow"])
        f["op_flow"].resize(f["op_flow"].len() + len(op_flows), axis = 0)
        f["op_flow"][-len(op_flows):] = op_flows
       

In [17]:
def opticalFlowDense(image_current, image_next):
    """
    Args:
        image_current : RGB image
        image_next : RGB image
    return:
        optical flow magnitude and angle and stacked in a matrix
    """
    image_current = np.array(image_current)
    image_next = np.array(image_next)
    gray_current = cv.cvtColor(image_current, cv.COLOR_RGB2GRAY)
    gray_next = cv.cvtColor(image_next, cv.COLOR_RGB2GRAY)
    flow = cv.calcOpticalFlowFarneback(gray_current, gray_next, None, 0.5, 1, 15, 2, 5, 1.3, 0)
    return flow


In [20]:
def dense_op4():
    op_flows = []
    count = 0
    vidcap = cv.VideoCapture(vidPath)
    success,frame1 = vidcap.read()
    frame1 = cv.cvtColor(frame1, cv.COLOR_BGR2RGB)
    frame1 = cv.resize(frame1, (0,0), fx=0.5, fy=0.5) 
    while success:
        if (count % 100 == 0) and count > 0:
            print(count)
        success,frame2 = vidcap.read()
        if success == True:
            frame2 = cv.cvtColor(frame2, cv.COLOR_BGR2RGB)
            frame2 = cv.resize(frame2, (0,0), fx=0.5, fy=0.5) 
            flow = opticalFlowDense(frame1, frame2)
            op_flows.append(flow)
            frame1 = frame2
            count+=1
        else:
            print("video reading completed")
            continue
    
    write_hdf5(hdf5_path4,np.array(op_flows))
    return count

def dense_op1():
    op_flows = []
    count = 0
    vidcap = cv.VideoCapture(vidPath)
    success,frame1 = vidcap.read()
    frame1 = cv.cvtColor(frame1, cv.COLOR_BGR2RGB)
    frame1 = cv.resize(frame1, (0,0), fx=0.5, fy=0.5) 
    while success:
        if (count % 100 == 0) and count > 0:
            print(count)
        success,frame2 = vidcap.read()
        if success == True:
            frame2 = cv.cvtColor(frame2, cv.COLOR_BGR2RGB)
            frame2 = cv.resize(frame2, (0,0), fx=0.5, fy=0.5) 

            frame1,frame2=augment(frame1,frame2)
            frame1,frame2=flipimage(frame1,frame2)
            flow = opticalFlowDense(frame1, frame2)

            op_flows.append(flow)

            frame1 = frame2
            count+=1
        else:
            print("video reading completed")
            continue
    
    write_hdf5(hdf5_path1,np.array(op_flows))
    return count

def dense_op3():
    op_flows = []
    count = 0
    vidcap = cv.VideoCapture(vidPath)
    success,frame1 = vidcap.read()
    frame1 = cv.cvtColor(frame1, cv.COLOR_BGR2RGB)
    frame1 = cv.resize(frame1, (0,0), fx=0.5, fy=0.5) 
    while success:
        if (count % 100 == 0) and count > 0:
            print(count)
        success,frame2 = vidcap.read()
        if success == True:
            frame2 = cv.cvtColor(frame2, cv.COLOR_BGR2RGB)
            frame2 = cv.resize(frame2, (0,0), fx=0.5, fy=0.5) 

            frame1,frame2=augment(frame1,frame2)
            flow = opticalFlowDense(frame1, frame2)

            op_flows.append(flow)

            frame1 = frame2
            count+=1
        else:
            print("video reading completed")
            continue
    
    write_hdf5(hdf5_path3,np.array(op_flows))
    return count

def dense_op2():
    op_flows = []
    count = 0
    vidcap = cv.VideoCapture(vidPath)
    success,frame1 = vidcap.read()
    frame1 = cv.cvtColor(frame1, cv.COLOR_BGR2RGB)
    frame1 = cv.resize(frame1, (0,0), fx=0.5, fy=0.5) 
    while success:
        if (count % 100 == 0) and count > 0:
            print(count)
        success,frame2 = vidcap.read()
        if success == True:
            frame2 = cv.cvtColor(frame2, cv.COLOR_BGR2RGB)
            frame2 = cv.resize(frame2, (0,0), fx=0.5, fy=0.5) 

            frame1,frame2=flipimage(frame1,frame2)
            flow = opticalFlowDense(frame1, frame2)

            op_flows.append(flow)

            frame1 = frame2
            count+=1
        else:
            print("video reading completed")
            continue
    
    write_hdf5(hdf5_path2,np.array(op_flows))
    return count


In [21]:
dense_op2()
dense_op1()
dense_op3()
dense_op4()

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

C:\Users\Nihar\miniconda3\envs\TensorFlow-GPU\lib\site-packages\ipykernel_launcher.py:2: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  


20399
<HDF5 dataset "op_flow": shape (1, 240, 320, 2), type "<f4">
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
174

20399